# Analysis of Job Ads and Skill Extraction

In [1]:
# Settings
import warnings
warnings.filterwarnings("ignore")

# Libraries
import os
import re
import time
import json
import boto3
import gzip
import ollama
import pandas as pd
from pprint import pprint
from io import BytesIO

from openai import OpenAI
from keybert import KeyBERT
from dotenv import load_dotenv
from transformers import pipeline

from sentence_transformers import SentenceTransformer, util

import torch

# Load spacy models (if not exist)
import spacy
import spacy.cli

def ensure_spacy_model(model_name):
    try:
        return spacy.load(model_name)
    except OSError:
        spacy.cli.download(model_name)
        return spacy.load(model_name)

nlp_de = ensure_spacy_model("de_core_news_sm")
nlp_en = ensure_spacy_model("en_core_web_sm")

# Display current working directory
print(os.getcwd())


/home/ec2-user/SageMaker/skill_framework/notebooks


## Read list with JobCloud professions

These are the official professions used in the search of www.jobs.ch

In [2]:
# Define bucket and key
bucket_name = 'jc-innosuisse'
object_key = 'ad_data/profession_synonyms_id_primary_map.csv'

# Initialize S3 client
s3_client = boto3.client('s3')

# Fetch object from S3
response = s3_client.get_object(Bucket=bucket_name, Key=object_key)

# Read CSV content directly using pandas
csv_content = response['Body'].read()
df_occjc = pd.read_csv(BytesIO(csv_content))

# Display basic information
print(f"DataFrame shape: {df_occjc.shape}")
print("\nColumn names:")
for col in df_occjc.columns:
    print(f"- {col}")

# Show sample data
print("\nSample data:")
df_occjc.head(10)

DataFrame shape: (2331, 4)

Column names:
- ful_id
- DE
- EN
- FR

Sample data:


,ful_id,DE,EN,FR
0,11000002,Assistent,Administrative Assistant,Assistant administratif
1,11000004,Team-Assistent,Team Assistant,Team Assistant
2,11000005,Allrounder,All-Rounder,Employé polyvalent
3,11000008,Assistent Betriebsleitung,Assistant Vice President,Adjoint au chef de département
4,11000009,Sachbearbeiter Back Office,Administrative Specialist,Administrative Expert
5,11000011,Büroangestellte,Office Clerk,Aide de bureau
6,11000013,Leiter Planung,Head of Planning,Directeur planification opérationnelle
7,11000014,Operativer Leiter,Chief Operating Officer,Chief Operation Officer
8,11000015,Datenerfasser,Data Entry Operator,Dactylographe
9,11000016,Assistent der Geschäftsleitung,Executive Assistant,Assistant de Direction


## Read job ads

This is a sample (10,000 obs) of job ads from JobClouds ad inventory with > 5 mio. job ads.

In [3]:
# Define bucket and key
bucket_name = 'jc-innosuisse'
object_key = 'ad_data/full_datapool_100000.json'
# object_key = 'ad_data/full_datapool_anonymized.json.gz'

# Initialize S3 client
s3_client = boto3.client('s3')

# Fetch object from S3
response = s3_client.get_object(Bucket=bucket_name, Key=object_key)

# Check if file is gzipped JSON or regular JSON
if object_key.endswith('.json.gz'):
    # Read and decompress the gzipped content
    with gzip.GzipFile(fileobj=BytesIO(response['Body'].read())) as gz:
        json_bytes = gz.read()
    # Load JSON data into pandas DataFrame
    data = json.loads(json_bytes.decode('utf-8'))
elif object_key.endswith('.json'):
    # For regular JSON, read directly
    json_bytes = response['Body'].read()
    data = json.loads(json_bytes.decode('utf-8'))
else:
    raise ValueError(f"Unsupported file format: {object_key}. Expected .json or .json.gz")

# Create DataFrame
df = pd.DataFrame(data)

# Show shape
print(df.shape)

# Show column names
for col in df.columns:
    print(col)

# Display DataFrame
df.head(2)

(10000, 27)
id
applicationEmail
title
leadText
text
html
preview
profession
placeOfWork
images
links
language
employmentTypes
employmentGrade
employmentPositions
baseSalary
locations
persons
attributes
platforms
companiesPublishedOn
company
requirements
updatedAt
_version
_links
_type


,id,applicationEmail,title,leadText,text,html,preview,profession,placeOfWork,images,...,persons,attributes,platforms,companiesPublishedOn,company,requirements,updatedAt,_version,_links,_type
0,5fed0746-a0d0-4559-8062-01df1ef952a0,rueti@ziwalig.ch,Supply Chain Manager,<b>Sie sind ein geschickter Verhandlungspartne...,<b>Ihre Aufgaben</b><br /><b></b>Mitglied im i...,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",None,Supply Chain Manager,None,[],...,[],"[{'key': 'LEGACY_JOB_ID', 'value': 3613149, '_...","[{'name': 'JOBS', 'publications': [{'type': 'S...",[{'id': 'b9133ccb-d568-441e-ad93-466f23db907f'...,"{'name': 'ZIWALIG AG Rüti', 'segment': 'PERSON...","{'languages': [{'language': 'de', 'level': 'FL...",2018-08-28T13:59:11+02:00,c39afe686a20b27b04ff662c59e080e6c04978c743630a...,{'self': {'href': '/job/5fed0746-a0d0-4559-806...,job
1,5104eb99-c228-4891-b147-4cc0269e4957,hr@bedag.ch,Solution Manager (w/m) mit Ausprägung Projekt-...,<p></p><p>Für unseren Bereich IT-Services in B...,<p>Zu Ihrem Aufgabengebiet gehen die Planung u...,"<!DOCTYPE html>\n<html lang=""de"">\n<head>\n <...",None,Solution Manager (w/m) mit Ausprägung Projekt-...,None,[],...,[],"[{'key': 'LEGACY_JOB_ID', 'value': 3613055, '_...","[{'name': 'JOBS', 'publications': [{'type': 'S...",[{'id': 'df1664bf-4df4-4647-8fe6-4297d7fc7cad'...,"{'name': 'Bedag Informatik AG', 'segment': 'SM...","{'languages': [], 'workExperiences': [{'profes...",2018-08-28T13:59:12+02:00,79a208f5bbda86619fe8214191057fcbf60f1748073195...,{'self': {'href': '/job/5104eb99-c228-4891-b14...,job


## Count 'id'

In [4]:
# Count unique IDs
df['id'].nunique()


10000

## Show 'leadText'

In [5]:
# Print sample title
df['leadText'][0]

'<b>Sie sind ein geschickter Verhandlungspartner!?</b><br><br>Unsere Mandantin ist ein weltweit marktführendes Unternehmen der High Tech-Industrie.'

## Show 'title'

In [6]:
# Count unique titles
print(df['title'].nunique(), "\n")

# Print first titles
for element in df['title'][:10]:
    print(element)

7737 

Supply Chain Manager
Solution Manager (w/m) mit Ausprägung Projekt-/Bid- & Proposal-Management
PMO Assistant
Développeur z/OS
Marketing Manager Konsumgüter
Administrator System AIX/Solaris
Elektrotechniker HF / eidg. dipl. Elektroinstallateur w/m
Einkäufer (Mechanik)
Dipl. Pflegefachfrau/-mann mit FA Intensivpflege 90%
Produkt Manager Food (m/w)


## Show 'text'

In [7]:
# Print sample text
df['text'][10]


'<p>In dieser Position beraten Sie unsere Kundschaft umfassend und kompetent von der ersten Kontaktaufnahme an über Versicherungsfragen bis hin zur Fahrzeugablieferung. Arbeiten wie Offerterstellung, Eintauschgeschäfte oder Akquirieren von Neukunden und Pflege des bestehenden Kundenstammes sind weitere Punkte in Ihrem Aufgabenbereich.</p><p>Sie verfügen entweder über eine erfolgreich abgeschlossene Ausbildung in der Automobilbranche mit einer kaufmännischen Weiterbildung oder über eine kaufmännische Ausbildung im technischen Bereich. Als erfahrener Verkaufsprofi erzielen Sie durch Ihren Einsatz und Enthusiasmus beste Verkaufsergebnisse und überzeugen im täglichen Kontakt mit unseren Kunden durch Ihr professionelles Auftreten.</p><p>Wir wünschen uns eine gewinnende, charismatische und kommunikative Persönlichkeit, die ihre Begeisterung für das Automobil wirkungsvoll auf die Kundschaft zu übertragen weiss.</p><p>Es erwartet Sie einen den Leistungen entsprechenden Lohn, eine moderne Infra

## Show 'profession'

In [8]:
# Count unique professions
print(df['profession'].nunique(), "\n")

# Print first professions 
for element in df['profession'][:10]:
    print(element)

7717 

Supply Chain Manager
Solution Manager (w/m) mit Ausprägung Projekt-/Bid- & Proposal-Management
PMO Assistant
Développeur z/OS
Marketing Manager Konsumgüter
Administrator System AIX/Solaris
Elektrotechniker HF / eidg. dipl. Elektroinstallateur w/m
Einkäufer (Mechanik)
Dipl. Pflegefachfrau/-mann mit FA Intensivpflege 90%
Produkt Manager Food (m/w)


## Show 'company' information

In [9]:
# Print sample company info 
df['company'][10]


{'name': 'Emil Frey AG',
 'segment': 'LARGE_FIRM',
 'address': {'street': 'Badenerstrasse 600',
  'postalCode': '8048',
  'city': 'Zürich',
  'countryCode': 'CH',
  'latitude': None,
  'longitude': None,
  '_type': 'address'},
 'images': [{'type': 'LOGO',
   'description': None,
   'url': 'https://img.jobs.ch/www/img/toplogos/5959.gif',
   'weight': 1,
   '_type': 'image'}],
 'attributes': [{'key': 'LEGACY_COMPANY_ID',
   'value': 5959,
   '_type': 'attribute'}],
 '_type': 'embeddedCompany'}

## Show 'requirements'

In [10]:
# Show structure of requirements
pprint(df['requirements'][1])

# Show first requirements from list
print("\n")
for element in df['requirements'][1]:
    print(element)


{'_type': 'requirements',
 'educations': [{'_type': 'education_requirement', 'level': 'UNIVERSITY'}],
 'languages': [],
 'workExperiences': [{'_type': 'work_experience_requirement',
                      'position': 'PROFESSIONAL_RESPONSIBILITY',
                      'profession': 'PROJECT_MANAGEMENT_ANALYSIS'}]}


languages
workExperiences
educations
_type


## Search for specific profession in job ads

In [11]:
# Check if term is in list
matching_entries = df[df['profession'].str.contains('.*Pflegefach.*', regex=True, na=False)]['profession'].tolist()

# Show occurences
print(len(matching_entries))

# Show matches
matching_entries

233


['Dipl. Pflegefachfrau/-mann mit FA Intensivpflege 90%',
 'Dipl. Pflegefachperson Temporär',
 'dipl. Pflegefachpersonal',
 'Dipl. Pflegefachperson HF für Kardiochirurgie',
 'Dipl. Pflegefachperson HF oder Höfa 1 zu 80%-90%',
 'Dipl. Pflegefachperson 60 - 100 % (104574)',
 'Flexibles Pflegefachpersonal zu einem 60-100% Pensum',
 'Dipl. Pflegefachfrau/-mann DNII / HF für temporäre Einsätze Altenpflege',
 'Dipl. Pflegefachfrau/-mann mit FA Notfallpflege für temporäre Einsätze',
 'Eine diplomierte Pflegefachfrau als Nachtwache 40-60%',
 'Dipl. Pflegefachfrau/-mann mit FA OPS oder TOA für temporäre Einsätze',
 'Dipl. Pflegefachfrau/-mann mit FA Intensivpflege, Arbeitspensum 80% -100%',
 'Dipl. Pflegefachfrau/-mann mit FA OPS oder TOA für temporäre Einsätze',
 'Pflegefachperson Neonatologie oder Expertin/Experte Intensivpflege',
 'Dipl. Pflegefachfrau/-mann mit FA Intensivpflege 70-100%',
 'Dipl. Pflegefachperson 50-60% bis Dezember 2011 für eine Spitex',
 'Dipl. Pflegefachfrau/mann für den 

## Extract skills

The goal here is to test different methods to extract the skills from the job describtions (field: text).
The methode are:  
- extracting skills through locally deployed LLM
- Extracting skills through GPT model (OpenAI API)
- Semantic search via sentence transformer based on ESCO - skills
- Extract named entities (no NER 'skills' available) using a pretrained NER model
- Unsupervised keyphrase extraction with keybert

First Fazit: 
- The highest accuracy provides the GPT model followed by a much smaller locally deployed LLM.
- The challenge with the GPT model is the price per token. 
- The challenge with the locally deployed LLM is the latency.

In [12]:
# Create subset of columns 
df_sub = df[['id', 'title', 'profession', 'text']]

# Show title
print("Title:", df_sub['title'][10])

# Show single text
df_sub['text'][10]

Title: Automobilverkäufer für die Marke Lancia


'<p>In dieser Position beraten Sie unsere Kundschaft umfassend und kompetent von der ersten Kontaktaufnahme an über Versicherungsfragen bis hin zur Fahrzeugablieferung. Arbeiten wie Offerterstellung, Eintauschgeschäfte oder Akquirieren von Neukunden und Pflege des bestehenden Kundenstammes sind weitere Punkte in Ihrem Aufgabenbereich.</p><p>Sie verfügen entweder über eine erfolgreich abgeschlossene Ausbildung in der Automobilbranche mit einer kaufmännischen Weiterbildung oder über eine kaufmännische Ausbildung im technischen Bereich. Als erfahrener Verkaufsprofi erzielen Sie durch Ihren Einsatz und Enthusiasmus beste Verkaufsergebnisse und überzeugen im täglichen Kontakt mit unseren Kunden durch Ihr professionelles Auftreten.</p><p>Wir wünschen uns eine gewinnende, charismatische und kommunikative Persönlichkeit, die ihre Begeisterung für das Automobil wirkungsvoll auf die Kundschaft zu übertragen weiss.</p><p>Es erwartet Sie einen den Leistungen entsprechenden Lohn, eine moderne Infra

## Extracting skills through locally deployed LLM

In [13]:
%%time

# Send prompt to the locally running LLM
response = ollama.chat(
    model='gemma3:1b',
    messages=[
        {
            'role': 'user',
            'content': (
                "Extrahiere technische und transversale Skills aus dem folgenden Text "
                "(kommagetrennt pro Zeile):\n\n" + df_sub['text'][10]
            ),
        }
    ]
)

# Extract content string
raw_text = response.message.content

# Function to format the code
def format_skills(text: str) -> str:
    text = text.replace("*", "-")
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]

    output_lines = []
    current_section = None

    for line in lines:
        clean_line = line.lower().replace("-", "").strip()
        
        if "technische skills" in clean_line:
            current_section = "Technische Skills"
            output_lines.append(f"  {current_section}:")
        elif "transversale skills" in clean_line:
            current_section = "Transversale Skills"
            output_lines.append(f"  {current_section}:")
        elif current_section and line.startswith("-"):
            output_lines.append(f"  {line}")

    return "\n".join(output_lines)

# Format and print
formatted_output = format_skills(raw_text)
print(formatted_output, "\n")


  Technische Skills:
  -   Automobilbranche
  -   Kaufmännische Weiterbildung
  -   Technische Ausbildung
  Transversale Skills:
  -   Verkauf
  -   Kommunikation
  -   Überzeugungskraft
  -   Kundenservice
  -   Persönliches Auftreten
  -   Sachliche Beratung
  -   Verhandlungsgeschäfte
  -   Kundenbeziehungen 

CPU times: user 3.23 ms, sys: 3.3 ms, total: 6.53 ms
Wall time: 22.6 s


## Extracting skills through GPT model (OpenAI API)

### Anonymize text

In [14]:
# Load spaCy's German model
nlp = spacy.load("de_core_news_sm")

# Function to anonymize text
def anonymize_text(text):
    doc = nlp(text)
    redacted_text = text
    spans_to_replace = []

    # --- Names and Company Names ---
    for ent in doc.ents:
        if ent.label_ == "PER":
            spans_to_replace.append((ent.start_char, ent.end_char, "[REDACTED NAME]"))
        elif ent.label_ == "ORG":
            spans_to_replace.append((ent.start_char, ent.end_char, "[REDACTED COMPANY]"))

    # Replace in reverse order
    for start, end, replacement in sorted(spans_to_replace, reverse=True):
        redacted_text = redacted_text[:start] + replacement + redacted_text[end:]

    # --- Phone Numbers ---
    phone_patterns = [
        r'(\+|00)?41[\s\-]?\d{2}([\s\-]?\d{2}){3}',
        r'\b\d{3}[\s\-]?\d{3}[\s\-]?\d{2}[\s\-]?\d{2}\b',
        r'(?:(?:\+|00)41\s?|0)\d{2}\s?\d{3}\s?\d{2}\s?\d{2}'
    ]
    for pattern in phone_patterns:
        redacted_text = re.sub(pattern, "[REDACTED PHONE]", redacted_text)

    # --- Email Addresses ---
    email_pattern = r'[\w\.-]+@[\w\.-]+\.\w+'
    redacted_text = re.sub(email_pattern, "[REDACTED EMAIL]", redacted_text)

    return redacted_text

# Call function
pos = 10
text = anonymize_text(df_sub['text'][pos])
print(df_sub['text'][pos], "\n")
print(text)

<p>In dieser Position beraten Sie unsere Kundschaft umfassend und kompetent von der ersten Kontaktaufnahme an über Versicherungsfragen bis hin zur Fahrzeugablieferung. Arbeiten wie Offerterstellung, Eintauschgeschäfte oder Akquirieren von Neukunden und Pflege des bestehenden Kundenstammes sind weitere Punkte in Ihrem Aufgabenbereich.</p><p>Sie verfügen entweder über eine erfolgreich abgeschlossene Ausbildung in der Automobilbranche mit einer kaufmännischen Weiterbildung oder über eine kaufmännische Ausbildung im technischen Bereich. Als erfahrener Verkaufsprofi erzielen Sie durch Ihren Einsatz und Enthusiasmus beste Verkaufsergebnisse und überzeugen im täglichen Kontakt mit unseren Kunden durch Ihr professionelles Auftreten.</p><p>Wir wünschen uns eine gewinnende, charismatische und kommunikative Persönlichkeit, die ihre Begeisterung für das Automobil wirkungsvoll auf die Kundschaft zu übertragen weiss.</p><p>Es erwartet Sie einen den Leistungen entsprechenden Lohn, eine moderne Infras

### Extract skills

In [15]:
%%time

# Load .env file
load_dotenv()

# Initialize OpenAI client using key from .env
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Prompt
prompt = (
    "Extrahiere technische und transversale Skills aus dem folgenden Text "
    "(kommagetrennt pro Zeile):\n\n" + text
)

# Response
response = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": prompt}],
    temperature=1,
)

# Extract and print the result
def print_skills_formatted(text):
    for section in text.strip().split("\n"):
        if ":" in section:
            category, skills = section.split(":", 1)
            skill_list = [s.strip().strip(".") for s in skills.split(",") if s.strip()]
            print(f"{category.strip()}:")
            for skill in skill_list:
                print(f"  - {skill}")
            print()

# Correct usage
response_text = response.choices[0].message.content
print_skills_formatted(response_text)


Technische Skills:
  - Beratung in Versicherungsfragen
  - Offerterstellung
  - Kundenakquise
  - Pflege des Kundenstammes
  - Ausbildung in der Automobilbranche
  - kaufmännische Weiterbildung
  - kaufmännische Ausbildung im technischen Bereich

Transversale Skills:
  - Kundenberatung
  - Verkaufsfähigkeiten
  - professionalles Auftreten
  - Kommunikationsfähigkeit
  - Charisma
  - Begeisterungsfähigkeit

CPU times: user 171 ms, sys: 23.3 ms, total: 195 ms
Wall time: 5.91 s


## Semantic search via sentence transformer based on ESCO - skills

### Select test case: Fachverkäufer für Kraftfahrzeuge

In [16]:
anonymize_text(df_sub['text'][1])

'<p>Zu Ihrem Aufgabengebiet gehen die Planung und Umsetzung von anspruchsvollen und komplexen Infrastrukturprojekten sowie von Angebots- und Verkaufsprojekten. Sie koordinieren alle Aktiviten im Rahmen dieser Projekte und stellen zusammen mit dem [REDACTED COMPANY] respektive Selling-Team den erfolgreichen Abschluss in qualitativer und zeitlicher Hinsicht sicher. Durch eine gute Ressourcen- und Budgetplanung sowie ein kontinuierliches Projekt-Controlling erfreuen Sie die Projektsponsoren.</p><p>In Angebots- und Verkaufsprojekten koordinieren Sie Selling-Teams von der Angebotserstellung bis zum Vertragsabschluss. Zusammen mit dem verantwortlichen Sales Manager erarbeiten Sie mit Unterstzung der [REDACTED COMPANY], den Head of Units sowie externen Partnern die Kundenlung und erstellen Angebots- und Vertragsdokumente.</p><p>F diese herausfordernde Tigkeit verfen Sie er einen Hochschulabschluss in Informatik (Uni, [REDACTED COMPANY], [REDACTED COMPANY]) und knen fundierte Erfahrungen in de

### Read ESCO data

In [17]:
# Read ESCO data
df_ski = pd.read_csv('./data/skills_de.csv')
df_occ = pd.read_csv('./data/occupations_de.csv')
df_osr = pd.read_csv('./data/occupationSkillRelations_de.csv')

# Select occupation-skill-relation (Fachverkäufer für Kraftfahrzeuge)
occ_uri = 'http://data.europa.eu/esco/occupation/6a163fef-dee8-4583-82b0-13b76d8a27b2'
df_osr_sub = df_osr.loc[df_osr['occupationUri'] == occ_uri]

# Select skills data
columns = ['conceptUri', 'skillType', 'reuseLevel', 'preferredLabel']
df_skills = df_ski.loc[df_ski['conceptUri'].isin(df_osr_sub['skillUri'])][columns]

# Select skills
skills = df_skills['preferredLabel']

# Show number of skills
label = df_occ.loc[df_occ['conceptUri'] == occ_uri, 'preferredLabel'].values[0]
print(f"Number of Skills for '{label}': {df_skills.shape[0]}")

# Show skillType and reuseLevel in crosstab
crosstab = pd.crosstab(df_skills['reuseLevel'], df_skills['skillType'])
print("\nSkill levels:")
print(crosstab)

# Show skills
df_skills

FileNotFoundError: [Errno 2] No such file or directory: './data/skills_de.csv'

### Create embeddings and calculate similarity

In [ ]:
%%time

# Load model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Select text from job ad
text = df_sub['text'][1]

# Encode text and skills
text_embedding = model.encode(text, convert_to_tensor=True)
skill_embeddings = model.encode(skills.tolist(), convert_to_tensor=True)

# Compute cosine similarity
cosine_scores = util.cos_sim(text_embedding, skill_embeddings)[0]

# Get top 10 scores and indices
top_indices = cosine_scores.argsort(descending=True)[:10]

# Print top 10 matching skills
for idx in top_indices:
    i = idx.item()
    print(f"{skills.iloc[i]}: {cosine_scores[i]:.2f}")

print("\n")

## Extract named entities using a pretrained NER model

In [ ]:
%%time

# Load a German NER pipeline with a multilingual or German model
ner = pipeline("ner", 
               model="xlm-roberta-large-finetuned-conll03-german", 
               aggregation_strategy="simple")

text = df_sub['text'][10]

results = ner(text)

for r in results:
    word = r.get('word', '<no word>')
    entity_group = r.get('entity_group', '<no entity_group>')
    score = r.get('score', 0.0)
    print(f"{word} - {entity_group} ({score:.2f})")

print("\n")

## Unsupervised keyphrase extraction with keybert

In [ ]:
%%time

# Load model
kw_model = KeyBERT(model='paraphrase-multilingual-MiniLM-L12-v2')

text = df_sub['text'][1]

keywords = kw_model.extract_keywords(text, 
                                    keyphrase_ngram_range=(1, 4), 
                                    stop_words=None,
                                    top_n=15, 
                                    use_maxsum=True)

# Sort keywords by score descending
keywords_sorted = sorted(keywords, key=lambda x: x[1], reverse=True)

for keyword, score in keywords_sorted:
    print(f"{keyword} ({score:.4f})")

print("\n")


## Profession matching: JobCloud profession list <-> profession names from job ads

The goal here is to match professions from different sources  
- Skills belong to a profession in the job ad inventory  
- If these skills are extracted, they must be matched to the professions in the professions list  
- JobClouds 'official' profession list has currently 2331 entries (de, en, fr)  
- Alternative profession lists are ESCO (3039 professions) or www.berufsberatung.ch (1876 professions)  

## Normalize the text

In [ ]:
# Function to normalize the text
def normalize(text):
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    return text

## Create semantic embeddings

In [ ]:
# Define model
model = SentenceTransformer("distiluse-base-multilingual-cased-v1")

# Normalize data
occ_labels = df_occjc['DE'].apply(normalize).tolist()
professions = df['profession'].apply(normalize).tolist()

# Create embeddings
occ_embeddings = model.encode(occ_labels, convert_to_tensor=True)
prof_embeddings = model.encode(professions, convert_to_tensor=True)

## Calculate similarities

In [ ]:
# Calculate similarities
similarities = util.cos_sim(prof_embeddings, occ_embeddings)

# Matchings
top_indices = torch.argmax(similarities, dim=1).tolist()
top_scores = torch.max(similarities, dim=1).values.tolist()

# Write mapping to data frame
df['matched_occupation'] = [df_occjc['DE'].iloc[i] for i in top_indices]
df['similarity'] = top_scores

# Show result
df[['profession', 'matched_occupation', 'similarity']][:20]

## Match professions

In [ ]:
# Number of top matches to extract
top_n = 5

# Compute top_n most similar occupations
top_results = torch.topk(similarities, k=top_n, dim=1)

# Convert indices and similarity scores to lists
indices = top_results.indices.tolist()
scores = top_results.values.tolist()

# Map each profession to its top_n most similar occupations
df['top_matches'] = [
    [(df_occjc['DE'].iloc[idx], round(score, 2)) for idx, score in zip(row_idx, row_scores)]
    for row_idx, row_scores in zip(indices, scores)]

# Display the top matches
for _, row in df[['profession', 'top_matches']].head(100).iterrows():
    print(f"Profession: {row['profession']}")
    print("Top Matches:")
    for label, score in row['top_matches']:
        print(f"  - {label} ({score})")
    print("-" * 40)